## Include libraries and load pretrained weak models

In [2]:
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
import pickle
import numpy as np
import sys
import sklearn.svm.classes
import pandas as pd
import os
#path constants
train_path = '../../data/train'
test_path = '../../data/test'

#type constants
vehicle_types = ['ZVe44', 'ZV573', 'ZV63d', 'ZVfd4', 'ZVa9c', 'ZVa78', 'ZV252']

#two label dataframes
train_label_df = pd.read_csv(train_path + '/train_label.csv', delimiter = ',', encoding = 'utf-8')
test_label_df = pd.read_csv(test_path + '/test_label.csv', delimiter = ',', encoding = 'utf-8')

sys.path.append(r'D:/ProgramData/Anaconda3/Lib/site-packages/sklearn/svm/')

cluster_n = 50

saved_model_path = '../SVM/ZVe44/'

svm_list = list()
for i in range(cluster_n):
    print(saved_model_path +'trained'+str(i)+'.pkl')
    with open(saved_model_path +'trained'+str(i)+'.pkl', 'rb') as file:
        pickle_model = pickle.load(file)
        svm_list.append(pickle_model)

../SVM/ZVe44/trained0.pkl
../SVM/ZVe44/trained1.pkl
../SVM/ZVe44/trained2.pkl
../SVM/ZVe44/trained3.pkl
../SVM/ZVe44/trained4.pkl
../SVM/ZVe44/trained5.pkl
../SVM/ZVe44/trained6.pkl
../SVM/ZVe44/trained7.pkl
../SVM/ZVe44/trained8.pkl
../SVM/ZVe44/trained9.pkl
../SVM/ZVe44/trained10.pkl
../SVM/ZVe44/trained11.pkl
../SVM/ZVe44/trained12.pkl
../SVM/ZVe44/trained13.pkl
../SVM/ZVe44/trained14.pkl
../SVM/ZVe44/trained15.pkl
../SVM/ZVe44/trained16.pkl
../SVM/ZVe44/trained17.pkl
../SVM/ZVe44/trained18.pkl
../SVM/ZVe44/trained19.pkl
../SVM/ZVe44/trained20.pkl
../SVM/ZVe44/trained21.pkl
../SVM/ZVe44/trained22.pkl
../SVM/ZVe44/trained23.pkl
../SVM/ZVe44/trained24.pkl
../SVM/ZVe44/trained25.pkl
../SVM/ZVe44/trained26.pkl
../SVM/ZVe44/trained27.pkl
../SVM/ZVe44/trained28.pkl
../SVM/ZVe44/trained29.pkl
../SVM/ZVe44/trained30.pkl
../SVM/ZVe44/trained31.pkl
../SVM/ZVe44/trained32.pkl
../SVM/ZVe44/trained33.pkl
../SVM/ZVe44/trained34.pkl
../SVM/ZVe44/trained35.pkl
../SVM/ZVe44/trained36.pkl
../SVM/ZVe4

In [17]:
model = svm_list[0]
print(model)
print(model.support_vectors_)

SVC(C=0.9, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)
[[0.09101846 0.1039147  0.11444814 ... 0.         0.         1.        ]
 [0.04438014 0.07589779 0.07438593 ... 0.         0.         1.        ]
 [0.04607944 0.08558117 0.10634989 ... 0.         0.         1.        ]
 ...
 [0.03700318 0.08280045 0.08988523 ... 0.         0.         1.        ]
 [0.03496786 0.08425569 0.07320605 ... 0.         0.         1.        ]
 [0.04344518 0.07684005 0.0637134  ... 0.         0.         1.        ]]


## load train data and labels and generate pooling score tensors

In [113]:
def getLabel(filename, label_df):
    idx = label_df.loc[label_df['sample_file_name'] == filename]
    return idx.iloc[0]['label']

def feature_tensor_gen(path, label_df, model_list):
#path: train_path or test_path
#vehicle_type: one string element under vehicle_types = ['ZVe44', 'ZV573', 'ZV63d', 'ZVfd4', 'ZVa9c', 'ZVa78', 'ZV252']
    
    #these are variables to calculate traversing progress (DO NOT CHANGE)
    counts_per_percent = int(len(os.listdir(path)) / 100)
    percentage_completion = 0
    counter = 0
    
    #pooling result from 50 weak learners then concatenated with the label
    feature_tensor = np.empty((0, cluster_n+1))
    
    for file in os.listdir(path):
        if file == 'clustered':
            continue
        sample_df = pd.read_csv(path + '/' + file, delimiter = ',', encoding = 'utf-8')

        # NO NEED TO CHANGE ANYTHING ABOVE
        # --------------------------------------------------------------------------
        label = getLabel(file, label_df)
        np_arr = sample_df.to_numpy()
        
        feature_vector = list()
        
        if np_arr.shape[0] == 0:
            continue
            
        for i in range(cluster_n):
            input_vector = np_arr[np_arr[:,-1] == i]
            
            if input_vector.shape[0] == 0:
                feature_vector.append(0.5)
                continue
                
            model = model_list[i]
            pooling_score = np.average(model.predict(input_vector[:,:-1]))
            feature_vector.append(pooling_score)
        
        feature_vector.append(label)
        feature_vector = np.array(feature_vector) 
        feature_tensor = np.append(feature_tensor, [feature_vector], axis=0)
        # --------------------------------------------------------------------------
        # NO NEED TO CHANGE ANYTHING BELOW
        
        #belows are to show traversing progress (DO NOT CHANGE)
        counter += 1
        if counter == counts_per_percent:
            counter = 0
            percentage_completion += 1
            print('traversing files under', path, ':', percentage_completion, "%", end="\r", flush=True)
    return feature_tensor

In [114]:
csv_path = '../../cleaned_data/train/ZVe44/clustered/'
feature_tensor = feature_tensor_gen(csv_path, train_label_df, svm_list)

In [115]:
feature_tensor.shape

(9335, 51)

In [263]:
x = feature_tensor[:, 0:50]
y = feature_tensor[:,50]

In [264]:
print(feature_tensor.shape)
print(x.shape)
print(y.shape)
np.savetxt("feature_tensor.csv", feature_tensor, delimiter=",")

(9335, 51)
(9335, 50)
(9335,)


## Logistic Regression Kernel to convert pooling scores into label

In [250]:
lrm = LogisticRegression(max_iter = 1000000, penalty = 'none', tol = 1e-10)
lrm.fit(x, y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000000,
                   multi_class='auto', n_jobs=None, penalty='none',
                   random_state=None, solver='lbfgs', tol=1e-10, verbose=0,
                   warm_start=False)

## Performance analysis

In [251]:
y_hat = lrm.predict(x)

In [252]:
from sklearn.metrics import accuracy_score
accuracy_score(y, y_hat)

0.5675415104445635

In [253]:
from sklearn import metrics

fpr, tpr, thresholds = metrics.roc_curve(y, y_hat, pos_label=1)
metrics.auc(fpr, tpr)

0.5662276948166338

In [269]:
label = y
label.shape
label = label.reshape((-1, 1))
print(label.shape)

(9335, 1)


[[0.01976482]
 [0.01982305]
 [0.01977486]
 [0.01978055]
 [0.01978864]
 [0.01982305]
 [0.01977089]
 [0.01979931]
 [0.01975786]
 [0.01983795]
 [0.01985996]
 [0.0197858 ]
 [0.01984314]
 [0.01979325]
 [0.01980368]
 [0.0198007 ]
 [0.01983609]
 [0.01979697]
 [0.01982864]
 [0.0198454 ]
 [0.0198305 ]
 [0.01978324]
 [0.01975599]
 [0.01981746]
 [0.01983422]
 [0.01978766]
 [0.01978471]
 [0.01979531]
 [0.01983981]
 [0.01975395]
 [0.01984726]
 [0.01981001]
 [0.01984092]
 [0.01982119]
 [0.01984167]
 [0.01984167]
 [0.01981004]
 [0.01984913]
 [0.0198454 ]
 [0.01984202]
 [0.01979138]
 [0.0198454 ]
 [0.0197858 ]
 [0.01983972]
 [0.01982305]
 [0.01984726]
 [0.01978711]
 [0.01975786]
 [0.01984112]
 [0.01983981]]
0.026031066
0.034922335
0.0419925
0.053561863
0.060846277
0.071344405
0.07798608
0.090305306
0.09898233
[[0.01767407]
 [0.0182461 ]
 [0.01778043]
 [0.01783616]
 [0.01791156]
 [0.01824641]
 [0.01773531]
 [0.0180208 ]
 [0.01760381]
 [0.01839719]
 [0.01861225]
 [0.01788488]
 [0.01844686]
 [0.01795616]

0.5106588
0.5106588
0.5106588
0.5106588
0.5106588
0.5106588
0.5106588
0.5106588
0.5106588
[[-0.00035672]
 [ 0.00456745]
 [ 0.00063883]
 [ 0.00114168]
 [ 0.00173896]
 [ 0.00456696]
 [ 0.00025742]
 [ 0.00269208]
 [-0.0009416 ]
 [ 0.00599171]
 [ 0.00789431]
 [ 0.00149216]
 [ 0.00640842]
 [ 0.00212975]
 [ 0.00302789]
 [ 0.00277737]
 [ 0.00585514]
 [ 0.00244856]
 [ 0.00510616]
 [ 0.00663421]
 [ 0.00534064]
 [ 0.00130941]
 [-0.00100275]
 [ 0.00427763]
 [ 0.00562601]
 [ 0.00160788]
 [ 0.00146761]
 [ 0.00236611]
 [ 0.00614018]
 [-0.00130326]
 [ 0.00684444]
 [ 0.00364569]
 [ 0.00621587]
 [ 0.00449425]
 [ 0.00626402]
 [ 0.00633315]
 [ 0.00373702]
 [ 0.00698126]
 [ 0.00662404]
 [ 0.00636374]
 [ 0.00200967]
 [ 0.00661485]
 [ 0.00149661]
 [ 0.00606143]
 [ 0.00457625]
 [ 0.00682487]
 [ 0.0015949 ]
 [-0.00080397]
 [ 0.00623897]
 [ 0.00611119]]
0.5106588
0.5106588
0.5106588
0.5106588
0.5106588
0.5106588
0.5106588
0.5106588
0.5106588
[[-1.8746826e-03]
 [ 3.4062923e-03]
 [-7.9769571e-04]
 [-2.5558640e-0

0.5106588
0.5106588
0.5106588
0.5106588
0.5106588
0.5106588
0.5106588
0.5106588
0.5106588
[[-0.01229361]
 [-0.004636  ]
 [-0.01060978]
 [-0.0097871 ]
 [-0.00894591]
 [-0.00464366]
 [-0.011181  ]
 [-0.00743413]
 [-0.01316533]
 [-0.0021916 ]
 [ 0.00086238]
 [-0.00935551]
 [-0.0015614 ]
 [-0.00832409]
 [-0.00690822]
 [-0.00724532]
 [-0.00250162]
 [-0.00783525]
 [-0.00374886]
 [-0.00120395]
 [-0.00334894]
 [-0.00954757]
 [-0.01320107]
 [-0.00490864]
 [-0.00288991]
 [-0.009227  ]
 [-0.00928516]
 [-0.00794892]
 [-0.00202575]
 [-0.01377713]
 [-0.00085457]
 [-0.00584494]
 [-0.00190807]
 [-0.00458912]
 [-0.00180606]
 [-0.00171635]
 [-0.00563041]
 [-0.00064895]
 [-0.00122175]
 [-0.00166918]
 [-0.00849848]
 [-0.00123148]
 [-0.00933951]
 [-0.00216487]
 [-0.00461137]
 [-0.00089658]
 [-0.00916062]
 [-0.01286407]
 [-0.00183947]
 [-0.00207388]]
0.5106588
0.5106588
0.5106588
0.5106588
0.5106588
0.5106588
0.5106588
0.5106588
0.5106588
[[-0.01339558]
 [-0.00549656]
 [-0.01164094]
 [-0.01078711]
 [-0.0099

KeyboardInterrupt: 

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])